In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dataset_path = '/content/drive/My Drive/dataset'

In [4]:
import os
import numpy as np
from PIL import Image
from skimage.feature import hog
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [5]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for class_folder in os.listdir(folder):
        class_path = os.path.join(folder, class_folder)
        if not os.path.isdir(class_path):
            continue
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            img = img.resize((64, 64))  # Resize to a fixed size

            # Extract HOG features
            img_array = np.array(img)
            hog_features = hog(img_array, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)

            images.append(hog_features)
            labels.append(class_folder)
    return images, labels

In [6]:
train_folder = os.path.join(dataset_path, 'train')
test_folder = os.path.join(dataset_path, 'test')

In [8]:
X_train, y_train = load_images_from_folder(train_folder)
X_test, y_test = load_images_from_folder(test_folder)

In [10]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [11]:
# Train a Logistic Regression model
logistic_regression = LogisticRegression(max_iter=1000, random_state=42)
logistic_regression.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [12]:
y_pred = logistic_regression.predict(X_test)

In [13]:
# KPIs
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [14]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")
print(classification_report(y_test, y_pred))

Accuracy: 59.29%
Precision: 59.30%
Recall: 59.29%
F1 Score: 0.59
              precision    recall  f1-score   support

           0       0.60      0.57      0.58        70
           1       0.59      0.61      0.60        70

    accuracy                           0.59       140
   macro avg       0.59      0.59      0.59       140
weighted avg       0.59      0.59      0.59       140

